In [86]:
import os
import cv2
import json
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
import chardet

In [87]:
NUM_CLASSES = 4
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32

In [88]:
# JSON 파일이 있는 폴더 경로
json_folder_path = r'C:\Users\hxxzi\Desktop\sol\label'

# 이미지 파일이 있는 폴더 경로
image_folder_path = r'C:\Users\hxxzi\Desktop\sol\image'

In [89]:
# 클래스별 이미지와 레이블을 저장할 리스트
train_images = []
train_labels = []
val_images = []
val_labels = []

In [90]:
emotions = ["happy", "anger", "panic", "sadness"]


In [91]:
def preprocess_image(image_path, image_name):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Image {image_name} could not be read.")
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = preprocess_input(image)
        return image
    except Exception as e:
        print(f"Error processing image {image_name}: {e}")
        return None


In [92]:
def process_data(emotion, data_type):
    images = []
    labels = []
    json_file_path = os.path.join(json_folder_path, data_type, f"{data_type}_{emotion}.json")
    # json_folder_path = r'C:\Users\hxxzi\Desktop\sol\label'
    image_file_path = os.path.join(image_folder_path, data_type, f"{emotion}")
    # image_folder_path = r'C:\Users\hxxzi\Desktop\sol\image\'
    #   C:\Users\hxxzi\Desktop\sol\image\train\happy\vpa6b16f59f4d6d160b6d35d714836b3d85735890cd5b48ee3b3987e8686dotmv.jpg.
    #  C:\Users\hxxzi\Desktop\sol\image\train\happy\

    with open(json_file_path, 'r', encoding='euc-kr') as f:
        data = json.load(f)

    # JSON 파일이 비어 있지 않은지 확인
    if not data:
        print(f"JSON 파일에 데이터가 없습니다: {json_file_path}")
        return [], []


    for item in data:
        image_name = item["filename"]
        image_path = os.path.join(image_file_path, image_name)
        

        # 이미지 파일 존재 여부 확인
        if not os.path.exists(image_path):
            print(f"Image {image_name} not found at {image_path}.")
            continue

        # 이미지 전처리
        xception_input = preprocess_image(image_path,image_name)
        if xception_input is None:
            continue

        # 데이터에 추가
        images.append(xception_input)
        labels.append(emotion)

    return images, labels


In [93]:
# 각 클래스에 대한 학습용(train) 데이터 처리
for emotion in emotions:
    train_images_emotion, train_labels_emotion = process_data(emotion, 'train')
    train_images.extend(train_images_emotion)
    train_labels.extend(train_labels_emotion)

Image 2tby14ba67fea06c15c1c19c15c47b0a7c90bbe3f0607657c630df1cc9a64yo1v.jpg not found at C:\Users\hxxzi\Desktop\sol\image\train\happy\2tby14ba67fea06c15c1c19c15c47b0a7c90bbe3f0607657c630df1cc9a64yo1v.jpg.
Image 0jsg06048a28fde3caad404f93dec397e5c579e83c1c2ced67c346c93d48dju4r.jpg not found at C:\Users\hxxzi\Desktop\sol\image\train\happy\0jsg06048a28fde3caad404f93dec397e5c579e83c1c2ced67c346c93d48dju4r.jpg.
Image zlye07fb710879977db578f793838be19b15b80543e98be869488885e9101yolo.jpg not found at C:\Users\hxxzi\Desktop\sol\image\train\happy\zlye07fb710879977db578f793838be19b15b80543e98be869488885e9101yolo.jpg.
Image yukr630c1d4d2527207bd8298f2037e0c041ecf88c689c25a273cfd86f859m2eb.jpg not found at C:\Users\hxxzi\Desktop\sol\image\train\happy\yukr630c1d4d2527207bd8298f2037e0c041ecf88c689c25a273cfd86f859m2eb.jpg.
Image xtlfac8896aa1888e03cd9859891933511415cbcf9b991f41c0b51976a93ahf4x.jpg not found at C:\Users\hxxzi\Desktop\sol\image\train\happy\xtlfac8896aa1888e03cd9859891933511415cbcf9b991

In [94]:
# 각 클래스에 대한 검증용(validation) 데이터 처리
for emotion in emotions:
    val_images_emotion, val_labels_emotion = process_data(emotion, 'val')
    val_images.extend(val_images_emotion)
    val_labels.extend(val_labels_emotion)

Image b14secfe2b1749c94c7cc563020df10e40372c7b9c8dbe793779c6c2cb272cznr.jpg not found at C:\Users\hxxzi\Desktop\sol\image\val\sadness\b14secfe2b1749c94c7cc563020df10e40372c7b9c8dbe793779c6c2cb272cznr.jpg.


In [95]:
train_images, train_labels = np.array(train_images), np.array(train_labels)
val_images, val_labels = np.array(val_images), np.array(val_labels)


In [96]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

In [97]:
train_labels = to_categorical(train_labels_encoded, NUM_CLASSES)
val_labels = to_categorical(val_labels_encoded, NUM_CLASSES)

In [98]:
# VGG16 모델 불러오기 및 커스텀 레이어 추가
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

In [99]:
for layer in base_model.layers:
    layer.trainable = False

In [100]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [101]:
model = Model(inputs=base_model.input, outputs=predictions)

In [102]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │         4,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,244,100 (58.15 MB)

 Trainable params: 529,412 (2.02 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [104]:
history = model.fit(train_images, train_labels, epochs=50, validation_data=(val_images, val_labels), batch_size=32)

Epoch 1/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 490s 3s/step - accuracy: 0.3582 - loss: 1.3793 - val_accuracy: 0.2793 - val_loss: 1.4289
Epoch 2/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 497s 3s/step - accuracy: 0.4162 - loss: 1.2515 - val_accuracy: 0.3194 - val_loss: 1.3860
Epoch 3/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 453s 3s/step - accuracy: 0.4635 - loss: 1.1779 - val_accuracy: 0.2860 - val_loss: 1.4442
Epoch 4/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 451s 3s/step - accuracy: 0.4974 - loss: 1.1371 - val_accuracy: 0.2955 - val_loss: 1.4699
Epoch 5/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 452s 3s/step - accuracy: 0.5236 - loss: 1.0762 - val_accuracy: 0.2993 - val_loss: 1.4499
Epoch 6/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 452s 3s/step - accuracy: 0.5647 - loss: 1.0239 - val_accuracy: 0.3337 - val_loss: 1.4726
Epoch 7/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 451s 3s/step - accuracy: 0.5903 - loss: 0.9555 - val_accuracy: 0.3146 - val_loss: 1.5222
Epoch 8/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 471s 3s/step - accuracy: 0.6183 - loss: 0.9174 - val_accu

In [105]:
# 모델 평가 (정확도 출력)
loss, accuracy = model.evaluate(val_images, val_labels)
print("정확도:", accuracy)

33/33 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.3267 - loss: 3.1514
정확도: 0.3203050494194031


In [106]:
# history 객체를 통해 평균 정확도 계산
average_accuracy = np.mean(history.history['val_accuracy'])
print("평균 검증 정확도:", average_accuracy)

평균 검증 정확도: 0.30873212575912473
